# Huggingface Endpoints

Hugging Face Hub은 12만 개 이상의 모델, 2만 개의 데이터셋, 5만 개의 데모 앱(Spaces)을 보유한 플랫폼으로, 모두 오픈 소스이며 공개적으로 사용 가능합니다. 이 온라인 플랫폼에서 사람들은 쉽게 협업하고 함께 머신러닝을 구축할 수 있습니다.

Hugging Face Hub은 또한 다양한 ML 애플리케이션을 구축하기 위한 다양한 엔드포인트를 제공합니다. 이 예제는 다양한 유형의 엔드포인트에 연결하는 방법을 보여줍니다.

특히, 텍스트 생성 추론은 Text Generation Inference에 의해 구동됩니다. 이는 매우 빠른 텍스트 생성 추론을 위해 맞춤 제작된 Rust, Python, gRPC 서버입니다.


## 허깅페이스 토큰 발급


허깅페이스(https://huggingface.co) 에 회원가입을 한 뒤, 아래의 주소에서 토큰 발급을 신청합니다.

- 토큰 발급주소: https://huggingface.co/docs/hub/security-tokens


## HuggingFace 모델 리스트

- 허깅페이스 LLM 리더보드: https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
- 모델 리스트: https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads


## Installation and Setup


`HuggingFaceEndpoint` 클래스를 사용하여 Hugging Face 엔드포인트와 상호 작용할 수 있습니다.

- `langchain_community.llms` 모듈에서 `HuggingFaceEndpoint` 클래스를 임포트합니다.
- `HuggingFaceEndpoint` 클래스를 사용하면 Hugging Face에서 호스팅되는 언어 모델과 통신할 수 있습니다.
- 이 클래스는 Hugging Face 엔드포인트의 URL과 필요한 인증 정보를 사용하여 초기화됩니다.
- 초기화된 `HuggingFaceEndpoint` 객체를 통해 프롬프트를 전송하고 언어 모델의 응답을 받을 수 있습니다.


In [12]:
from langchain_community.llms import HuggingFaceEndpoint

SchemaError: Invalid Schema:
model.config.extra_fields_behavior
  Input should be 'allow', 'forbid' or 'ignore' [type=literal_error, input_value=<Extra.forbid: 'forbid'>, input_type=Extra]
    For further information visit https://errors.pydantic.dev/2.7/v/literal_error

사용하기 위해서는 Python의 `huggingface_hub` [패키지를 설치](https://huggingface.co/docs/huggingface_hub/installation)해야 합니다.


- `huggingface_hub` 라이브러리를 최신 버전으로 업그레이드합니다.


In [11]:
%pip install --upgrade --quiet huggingface_hub


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


아래의 코드를 실행하여 발급 받은 허깅페이스 토큰을 인증합니다.


In [13]:
from huggingface_hub import login

login()

이 코드는 Hugging Face Hub API 토큰을 안전하게 입력받아 `HUGGINGFACEHUB_API_TOKEN` 변수에 저장하는 역할을 합니다. 토큰은 Hugging Face Hub의 API 추론 기능을 사용하기 위해 필요합니다.


- 환경 변수 `HUGGINGFACEHUB_API_TOKEN`을 설정합니다.
  - `os.environ` 딕셔너리를 사용하여 `"HUGGINGFACEHUB_API_TOKEN"` 키에 `HUGGINGFACEHUB_API_TOKEN` 변수의 값을 할당합니다.


In [14]:
import os
from dotenv import load_dotenv

load_dotenv()

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

## Prepare Examples

예시 데이터를 준비하는 과정입니다.


`HuggingFaceEndpoint` 클래스를 사용하여 Hugging Face 엔드포인트와 상호 작용할 수 있습니다.

- `langchain_community.llms` 모듈에서 `HuggingFaceEndpoint` 클래스를 임포트합니다.
- `HuggingFaceEndpoint` 클래스를 사용하면 Hugging Face에서 호스팅되는 언어 모델과 통신할 수 있습니다.
- 이 클래스는 Hugging Face 엔드포인트의 URL과 필요한 인증 정보를 사용하여 초기화됩니다.
- 초기화된 `HuggingFaceEndpoint` 객체를 통해 프롬프트를 전송하고 언어 모델의 응답을 받을 수 있습니다.


In [16]:
from langchain_community.llms import HuggingFaceEndpoint

SchemaError: Invalid Schema:
model.config.extra_fields_behavior
  Input should be 'allow', 'forbid' or 'ignore' [type=literal_error, input_value=<Extra.forbid: 'forbid'>, input_type=Extra]
    For further information visit https://errors.pydantic.dev/2.7/v/literal_error

- `question` 변수에 "1994년 FIFA 월드컵에서 우승한 팀은 어디인가요?"라는 질문을 할당합니다.
- `template` 변수에 질문과 답변 형식을 지정하는 템플릿 문자열을 할당합니다.
  - 템플릿에는 `{question}` 플레이스홀더가 포함되어 있습니다.
  - 답변 부분에는 "단계별로 생각해 보겠습니다."라는 문구가 포함되어 있습니다.
- `PromptTemplate.from_template()` 메서드를 사용하여 `template`을 기반으로 `PromptTemplate` 객체인 `prompt`를 생성합니다.
  - 이를 통해 질문을 템플릿에 삽입하여 프롬프트를 생성할 수 있습니다.


In [17]:
from langchain.prompts import PromptTemplate

template = """Please answer the following questions concisely.
QUESTION: {question}

ANSWER: """

prompt = PromptTemplate.from_template(template)

## Examples

다음은 무료 [Serverless Endpoints](https://huggingface.co/docs/api-inference/index) API의 `HuggingFaceEndpoint` 통합에 액세스하는 방법의 예시입니다.


- `repo_id` 변수에 "mistralai/Mistral-7B-Instruct-v0.2" 모델의 저장소 ID를 할당합니다.
- `HuggingFaceEndpoint`를 사용하여 `llm` 객체를 생성합니다.
  - `repo_id`로 지정된 모델을 사용합니다.
  - `max_length`를 128로 설정하여 생성할 최대 토큰 수를 제한합니다.
  - `temperature`를 0.5로 설정하여 생성 결과의 다양성을 조절합니다.
  - `token`에 `HUGGINGFACEHUB_API_TOKEN`을 전달하여 인증합니다.
- `LLMChain`을 사용하여 `llm_chain` 객체를 생성합니다.
  - `prompt` 변수에 할당된 프롬프트를 사용합니다.
  - `llm` 매개변수에 이전에 생성한 `llm` 객체를 전달합니다.
- `llm_chain.run(question)`을 호출하여 주어진 질문에 대한 답변을 생성합니다.
- 생성된 답변을 출력합니다.


In [19]:
from langchain.chains import LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 사용할 모델의 저장소 ID를 설정합니다.
# repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
# repo_id = "google/gemma-7b"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,  # 모델 저장소 ID를 지정합니다.
    max_new_tokens=256,  # 생성할 최대 토큰 길이를 설정합니다.
    temperature=0.1,  # 샘플링 온도를 설정합니다. 값이 높을수록 더 다양한 출력을 생성합니다.
    callbacks=[StreamingStdOutCallbackHandler()],  # 콜백을 설정합니다.
    streaming=True,  # 스트리밍을 사용합니다.
)

# LLMChain을 초기화하고 프롬프트와 언어 모델을 전달합니다.
llm_chain = LLMChain(prompt=prompt, llm=llm)
# 질문을 전달하여 LLMChain을 실행하고 결과를 출력합니다.
response = llm_chain.invoke(
    {"question": "Please tell me top 5 places to visit in Seoul, Korea."}
)
print(response)

SchemaError: Invalid Schema:
model.config.extra_fields_behavior
  Input should be 'allow', 'forbid' or 'ignore' [type=literal_error, input_value=<Extra.forbid: 'forbid'>, input_type=Extra]
    For further information visit https://errors.pydantic.dev/2.7/v/literal_error

## Dedicated Endpoint

무료 서버리스 API를 사용하면 솔루션을 빠르게 구현하고 반복할 수 있지만, 로드가 다른 요청과 공유되기 때문에 대용량 사용 사례에서는 속도 제한이 있을 수 있습니다.

엔터프라이즈 워크로드의 경우, [Inference Endpoints - Dedicated](https://huggingface.co/inference-endpoints/dedicated)를 사용하는 것이 가장 좋습니다.

이를 통해 더 많은 유연성과 속도를 제공하는 완전 관리형 인프라에 액세스할 수 있습니다.

이러한 리소스에는 지속적인 지원과 가동 시간 보장은 물론 AutoScaling과 같은 옵션도 포함됩니다.


![](./images/hugging-face-endpoint.png)


- `your_endpoint_url` 변수에 Inference Endpoint의 URL을 설정합니다.


In [20]:
# Inference Endpoint URL을 아래에 설정합니다.
your_endpoint_url = "https://qkryokt2o80cnb8u.us-east-1.aws.endpoints.huggingface.cloud"

- `HuggingFaceEndpoint` 클래스를 사용하여 `llm` 객체를 생성합니다.
  - `endpoint_url` 매개변수에는 Hugging Face 엔드포인트의 URL을 전달합니다.
  - `max_new_tokens` 매개변수는 생성할 최대 토큰 수를 지정합니다.
  - `top_k` 매개변수는 확률 기반 샘플링에 사용할 상위 k개 토큰을 지정합니다.
  - `top_p` 매개변수는 누적 확률 기반 샘플링에 사용할 상위 확률 임계값을 지정합니다.
  - `typical_p` 매개변수는 전형적인 확률 기반 샘플링에 사용할 확률 임계값을 지정합니다.
  - `temperature` 매개변수는 샘플링 과정에서의 무작위성을 조절합니다.
  - `repetition_penalty` 매개변수는 반복 패널티를 조절하여 반복되는 단어나 구문을 방지합니다.
- `llm` 객체를 호출하여 "#QUESTION: 대한민국의 수도는 어디인가요?\n\n#ANSWER:" 라는 질문을 전달하면, 해당 질문에 대한 답변을 생성합니다.


In [29]:
llm = HuggingFaceEndpoint(
    # 엔드포인트 URL을 설정합니다.
    endpoint_url=f"{your_endpoint_url}",
    # 생성할 최대 토큰 수를 설정합니다.
    max_new_tokens=512,
    # 상위 K개의 토큰을 선택합니다.
    top_k=10,
    # 누적 확률이 top_p에 도달할 때까지 토큰을 선택합니다.
    top_p=0.95,
    # typical_p 확률 이상의 토큰만 선택합니다.
    typical_p=0.95,
    # 샘플링 온도를 설정합니다. 낮을수록 더 결정적입니다.
    temperature=0.01,
    # 반복 패널티를 설정합니다. 높을수록 반복을 줄입니다.
    repetition_penalty=1.03,
)
# 주어진 프롬프트에 대해 언어 모델을 실행합니다.
llm.invoke(input="#QUESTION: 대한민국의 수도는 어디인가요?\n\n#ANSWER:")

' 서울특별시입니다.\n대한민국은 1948년에 건국된 나라이다. 그런데 이때부터 지금까지의 행정수도로서 기능을 해온 곳이 바로 세종 특별자치시다. 하지만, 실제적인 국가적 권력과 의사결정권 등 모든 것이 모여있는 진짜 \'수도\'라고 한다면 당연히 우리나라는 #서울특별시 라고 할 것이다. 그리고 많은 국민들이 알고 있는 것처럼 정부청사와 대통령 집무실 및 관저를 비롯해서 주요 공공기관들 모두 다 여기에 있다. 또한 교육이나 문화예술 분야에서 가장 앞선 도시이며 경제활동 역시 활발하게 일어나는 지역으로써 명실상부 한국 최대의 중심지역임에는 누구도 부인하지 못할것 같다.\u200b그런 의미에서는 사실 상징성만 보더래도 "우리나라"하면 제일 처음 머릿속에 생각날 만큼 중요하고 자랑스러운 장소이고 동네(?)같다는 느낌마저 들게 하는곳이기도 한듯하다. 물론 다른 광역시들의 입장에서는 조금 서글픈 이야기겠으나 말이다...^^;\u200b \u200b아무튼 오늘 소개해드릴 내용은 위 질문 속 단골 손님중 하나인 대한 민국의 위치 관련 정보 입니다~\u200b\u200b \u200b\u200b먼저 아시아(Asia)쪽 사진들을 살펴보도록 합시다!\u200b\u200b\u200b위키백과 참조\u200b\u200b\u200b\u200b첫번째 이미지 출처 : https://commons.wikimedia.org/wiki/%EB%AF%B8_%EC%A0%9C2_(South_Korea)\u200b미주한국재단 제공사진https://commons.wikimedia.org/wiki 미주의 여러 재단 홈페이지에도 들어가봤는데 마땅한 자료 찾기가 너무 힘듭니다ㅠㅜ;;혹 좋은 링크 있으면 추천좀 해주세요~~!!\u200b\u200b\u200b\u200b\u200b\u200b\u200b두 번째 이미지출처 : commons.wikimedia.orghttps://commons.wikimedia.org/wiki/?q=south+korea&tbm=isch&source=iu&ictx=1&fir=-675324

HuggingFaceEndpoint를 사용하여 스트리밍 방식으로 텍스트를 생성하는 예제입니다.

- `HuggingFaceEndpoint` 클래스를 사용하여 `llm` 객체를 생성합니다.
  - `endpoint_url` 매개변수에 HuggingFace 엔드포인트 URL을 지정합니다.
  - 다양한 생성 매개변수를 설정합니다: `max_new_tokens`, `top_k`, `top_p`, `typical_p`, `temperature`, `repetition_penalty`.
  - `streaming` 매개변수를 `True`로 설정하여 스트리밍 모드를 활성화합니다.
- `StreamingStdOutCallbackHandler`를 콜백으로 사용하여 생성된 텍스트를 실시간으로 출력합니다.
- `llm` 객체를 호출하여 프롬프트에 대한 텍스트를 생성합니다.

이 예제는 HuggingFace 엔드포인트를 사용하여 텍스트 생성을 수행하고, 생성 과정을 실시간으로 스트리밍하는 방법을 보여줍니다.


In [28]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    # 엔드포인트 URL을 설정합니다.
    endpoint_url=f"{your_endpoint_url}",
    # 생성할 최대 토큰 수를 설정합니다.
    max_new_tokens=512,
    # top_k 샘플링 기법에서 고려할 최상위 토큰 수를 설정합니다.
    top_k=10,
    # top_p 샘플링 기법에서 누적 확률 임계값을 설정합니다.
    top_p=0.95,
    # typical_p 샘플링 기법에서 누적 확률 임계값을 설정합니다.
    typical_p=0.95,
    # 샘플링 온도를 설정합니다. 낮을수록 더 결정적인 출력을 생성합니다.
    temperature=0.01,
    # 반복 패널티를 설정하여 동일한 토큰의 반복을 제어합니다.
    repetition_penalty=1.03,
    # 스트리밍 모드를 활성화합니다.
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# 질문을 입력하고 스트리밍 콜백 핸들러를 사용하여 실시간으로 출력을 확인합니다.
llm.invoke(
    input="#QUESTION: 대한민국 경기도를 여행한다면 꼭 가봐야할 곳 5군데를 추천해 주세요.n\n#ANSWER:",
)

' 안녕하세요? 저는 한국관광공사에서 운영하는 트래블리더로 활동하고 있는 대학생입니다! 제가 소개시켜드릴 관광지는 바로 수원화성과 화성행궁, 그리고 광교호수공원인데요~ 지금부터 그 이유와 함께 설명해 드리도록 하겠습니다^^ 첫째로 들려볼 장소는 세계문화유산으로 등록되어있는 \'수원화성\' 입니다!! 우리나라의 성곽 중 가장 아름답다고 평가받고 있기 때문에 많은 분들이 찾아주십니다~~ 특히 야경이 정말 예쁘다는 사실~! 두번째 코스는 조선왕조 최대규모 행차였던 어머니에 대한 효심을 담은 왕의 길인 "화성어장" 을 걸으며 역사적 의미도 느껴보며 산책하기 좋은곳 인것 같네ㅎㅎ 마지막 세 번째코스는 호수위 공중정원 으로 불리는 ‘광교호수공원’ 입다!!! 자연친화적인 휴식처이며 다양한 문화예술프로그램들을 접하거나 참여 할 수 있어서 더욱 매력만점이고 사람들의 발길이 계속해서 향하게 되는 것같습니당^0^* 이상 간단하지만 알찬 정보들 잘 보셨나용?? 여러분께 도움되는 포스트였다면 좋겠구 앞으로 열심히 노력하여 유익한 글 많이 올리도록 약속하겟슴돠~~~ 감사합니다♥<eos>'